In [ ]:
!nvidia-smi

Mon Mar  4 22:34:51 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   58C    P0              29W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [ ]:
!pip -q install git+https://github.com/huggingface/transformers # need to install from github
# !pip install -q datasets loralib sentencepiece
!pip -q install bitsandbytes accelerate xformers einops
!pip -q install langchain
!pip install --upgrade --quiet  langchain-pinecone langchain
!pip install --upgrade --quiet  langchain sentence_transformers

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [ ]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    A token is already saved on your machine. Run `huggingface-cli whoami` to get more information or `huggingface-cli logout` if you want to log out.
    Setting a new token will erase the existing one.
    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Token: 
Add token as git credential? (Y/n) y
Token is valid (permission: write).
Cannot authenticate through git-crede

In [ ]:
!huggingface-cli whoami

srija10


In [ ]:
from langchain_community.embeddings import HuggingFaceEmbeddings

In [ ]:
model_langchain = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-l6-v2")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"
!pip install -U sentence-transformers
from sentence_transformers import SentenceTransformer

In [ ]:
model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

In [ ]:
from pinecone import Pinecone
from pinecone import PodSpec

api_key = "c46c6ae0-3403-455d-9b0c-97ece122839e"
environment = "gcp-starter"

pc = Pinecone(api_key=api_key)
spec = PodSpec(environment=environment)

In [ ]:
index_name = "courses-indexes"

index = pc.Index(index_name)

# view index stats
index.describe_index_stats()

{'dimension': 384,
 'index_fullness': 0.00221,
 'namespaces': {'': {'vector_count': 221}},
 'total_vector_count': 221}

In [ ]:
from langchain.vectorstores import Pinecone

text_field = "text"  # the metadata field that contains our text

# initialize the vector store object
vectorstore = Pinecone(
    index, model_langchain, text_field
)

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.vectorstores.pinecone.Pinecone` was deprecated in langchain-community 0.0.18 and will be removed in 0.2.0. An updated version of the class exists in the langchain-pinecone package and should be used instead. To use it run `pip install -U langchain-pinecone` and import as `from langchain_pinecone import Pinecone`.
  warn_deprecated(


In [ ]:
query = "The blockchain stack has five layers: decentralized applications, smart contracts, protocol"
results = vectorstore.similarity_search_with_score(query, k=1)
print(results)

for result in results:
    print(result)
    doc, score = result
    print(f"Document: {doc}")
    print(f"Similarity Score: {score}")
    print("-" * 20)

[(Document(page_content='CSE 509LEC - Advanced Blockchain ConceptsThe blockchain stack has five layers: decentralized applications, smart contracts, protocol, operating system, and network layers. This course focuses on the blockchain protocol layer, the support provided by the layers below it, and the algorithms and techniques supporting its design and implementation. Topics include Bitcoin and Ethereum blockchain protocols, state and storage management; consensus algorithms: scalability: side channel, block size, sharding, network-layer solutions; Universal digital identity; Confidentiality, security, and privacy methods: zero-knowledge proofs. Fungible and non-fungible tokens; Protocol improvement methods. This course is dual listed with CSE 409. Credits: 3'), 0.681824923)]
(Document(page_content='CSE 509LEC - Advanced Blockchain ConceptsThe blockchain stack has five layers: decentralized applications, smart contracts, protocol, operating system, and network layers. This course focu

In [ ]:
extracted_doc = results[0][0]
extracted_doc = str(extracted_doc)

In [ ]:
content = extracted_doc.split('=', 1)[1]
print(content)

'CSE 509LEC - Advanced Blockchain ConceptsThe blockchain stack has five layers: decentralized applications, smart contracts, protocol, operating system, and network layers. This course focuses on the blockchain protocol layer, the support provided by the layers below it, and the algorithms and techniques supporting its design and implementation. Topics include Bitcoin and Ethereum blockchain protocols, state and storage management; consensus algorithms: scalability: side channel, block size, sharding, network-layer solutions; Universal digital identity; Confidentiality, security, and privacy methods: zero-knowledge proofs. Fungible and non-fungible tokens; Protocol improvement methods. This course is dual listed with CSE 409. Credits: 3'


In [ ]:

from transformers import pipeline
from transformers import AutoTokenizer
import torch
# model = "meta-llama/Llama-2-7b-chat-hf"
# model = 'google/flan-t5-large'
model = 'meta-llama/Llama-2-7b-hf'
tokenizer = AutoTokenizer.from_pretrained(model, use_auth_token=True,low_cpu_mem_usage=True)
llama_pipeline = pipeline(
    "text-generation",  # LLM task
    model=model,
    torch_dtype=torch.float16,
    device_map="auto",
)

# flan_pipeline = pipeline("text2text-generation", model="google/flan-t5-small", torch_dtype=torch.float16, device_map="auto")

/usr/local/lib/python3.10/dist-packages/transformers/models/auto/tokenization_auto.py:728: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
def get_response(prompt: str) -> None:
    """
    Generate a response from the Llama model.

    Parameters:
        prompt (str): The user's input/question for the model.

    Returns:
        None: Prints the model's response.
    """
    sequences = llama_pipeline(
        prompt,
        do_sample=True,
        top_k=1,
        num_return_sequences=1,
        eos_token_id=tokenizer.eos_token_id,
        max_length=1024,
        truncation=True,
    )
    print(sequences)
    print("Chatbot:", sequences[0]['generated_text'])

In [ ]:
get_response("Hello, how are you?")

[{'generated_text': "Hello, how are you?\nI'm fine, thank you. How are you?\nI'm fine, thank you.\nI'm fine, thank you. How are you?\nI'm fine, thank you. How are you?\nI'm fine, thank you. How are you?\nI'm fine, thank you. How are you?\nI'm fine, thank you. How are you?\nI'm fine, thank you. How are you?\nI'm fine, thank you. How are you?\nI'm fine, thank you. How are you?\nI'm fine, thank you. How are you?\nI'm fine, thank you. How are you?\nI'm fine, thank you. How are you?\nI'm fine, thank you. How are you?\nI'm fine, thank you. How are you?\nI'm fine, thank you. How are you?\nI'm fine, thank you. How are you?\nI'm fine, thank you. How are you?\nI'm fine, thank you. How are you?\nI'm fine, thank you. How are you?\nI'm fine, thank you. How are you?\nI'm fine, thank you. How are you?\nI'm fine, thank you. How are you?\nI'm fine, thank you. How are you?\nI'm fine, thank you. How are you?\nI'm fine, thank you. How are you?\nI'm fine, thank you. How are you?\nI'm fine, thank you. How a

In [ ]:
query = "What is taught in Advanced Block chain Technology?"
results = vectorstore.similarity_search_with_score(query, k=1)
print(str(results[0][0]).split('=', 1)[1])
context = str(results[0][0]).split('=', 1)[1]


'CSE 526LEC - Blockchain Application DevelopmentThis course is intended for students interested in learning about the blockchain technology and in developing applications using the blockchain concepts. Topics include: blockchain peer-to-peer transactions, blocks of transactions, chain of blocks, chain integrity, and structure and operational details of a blockchain. Blockchain programming using smart contracts and decentralized application (Dapp) development will be discussed in detail using hands-on programming. Bitcoin and Ethereum blockchain protocols will be used to illustrate the various concepts, Solidity language for smart contract development, Remix and Truffle environments for Dapp development. This course is dual-listed with CSE 426.Credits: 3'


In [ ]:
chatbot = get_response(query + context)

[{'generated_text': "What is taught in Advanced Block chain Technology?'CSE 526LEC - Blockchain Application DevelopmentThis course is intended for students interested in learning about the blockchain technology and in developing applications using the blockchain concepts. Topics include: blockchain peer-to-peer transactions, blocks of transactions, chain of blocks, chain integrity, and structure and operational details of a blockchain. Blockchain programming using smart contracts and decentralized application (Dapp) development will be discussed in detail using hands-on programming. Bitcoin and Ethereum blockchain protocols will be used to illustrate the various concepts, Solidity language for smart contract development, Remix and Truffle environments for Dapp development. This course is dual-listed with CSE 426.Credits: 3'CSE 527LEC - Blockchain Application DevelopmentThis course is intended for students interested in learning about the blockchain technology and in developing applicat

In [ ]:

!pip install streamlit

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 25.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 195.4/195.4 kB 23.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 55.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.0/83.0 kB 12.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 9.6 MB/s eta 0:00:00


In [ ]:
%%writefile app.py

Writing app.py


In [ ]:
!wget -q -O - ipv4.icanhazip.com

35.227.172.12


In [ ]:
torch.cuda.empty_cache()
!streamlit run app.py & npx localtunnel --port 8501

[..................] | fetchMetadata: sill resolveWithNewModule localtunnel@2.0


  You can now view your Streamlit app in your browser.

  Network URL: http://172.28.0.12:8501
  External URL: http://35.227.172.12:8501

npx: installed 22 in 6.941s
your url is: https://cuddly-moles-stick.loca.lt


In [ ]:
# from langchain_core.prompts import PromptTemplate
# template = """
#               Write a concise summary of the following text delimited by
#               triple backquotes.
#               Return your response in bullet points which covers the key
#               points of the text.
#               ```{text}```
#               BULLET POINT SUMMARY:
#            """
# prompt = PromptTemplate(template=template, input_variables=["text"])


In [ ]:
# from transformers import AutoTokenizer
# import transformers
# import torch
# from langchain.chains import RetrievalQA
# from langchain import LLMChain
# from langchain import HuggingFacePipeline

# class RAGPipelineWrapper:
#     def __init__(self, model, vectorstore):
#         self.model = model
#         self.vectorstore = vectorstore
#         self.tokenizer = AutoTokenizer.from_pretrained(model, use_auth_token=True)
#         self.pipeline = transformers.pipeline(
#             "text-generation", #task
#             model=model,
#             tokenizer=self.tokenizer,
#             torch_dtype=torch.bfloat16,
#             trust_remote_code=True,
#             device_map="auto",
#             max_length=1000,
#             do_sample=True,
#             top_k=1,
#             num_return_sequences=1,
#             eos_token_id=self.tokenizer.eos_token_id
#         )
#         self.llm = HuggingFacePipeline(pipeline=self.pipeline, model_kwargs={'temperature':0})
#         self.rag_pipeline = RetrievalQA.from_chain_type(
#             llm=self.llm,
#             chain_type='stuff',
#             retriever=vectorstore.as_retriever(),
#             return_source_documents=True
#         )

#     def generate_response(self, prompt):
#         return self.rag_pipeline(prompt)

